In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context="notebook", style="darkgrid", palette="deep", font="sans-serif", font_scale=1, color_codes=True)

In [3]:
data= pd.read_csv("train.csv")

In [4]:
data

,Id,Province/State,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,33.0000,65.0000,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,33.0000,65.0000,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,33.0000,65.0000,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,33.0000,65.0000,2020-01-26,0.0,0.0
...,...,...,...,...,...,...,...,...
17603,26377,NaN,Zambia,-15.4167,28.2833,2020-03-19,2.0,0.0
17604,26378,NaN,Zambia,-15.4167,28.2833,2020-03-20,2.0,0.0
17605,26379,NaN,Zambia,-15.4167,28.2833,2020-03-21,2.0,0.0
17606,26380,NaN,Zambia,-15.4167,28.2833,2020-03-22,3.0,0.0


In [5]:
data["Date"].max() ## 2020-03-20
data = data[data["Date"]<="2020-03-11"]

In [6]:
data["Date"].max() ## 2020-03-11

'2020-03-11'

In [7]:
data["Date"] = data["Date"].apply(lambda x: x.replace("-",""))
data["Date"]  = data["Date"].astype(int)

D:\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
data = data.drop(['Province/State'],axis=1)
data = data.dropna()
data.isnull().sum()

Id                0
Country/Region    0
Lat               0
Long              0
Date              0
ConfirmedCases    0
Fatalities        0
dtype: int64

In [9]:
data.head()

,Id,Country/Region,Lat,Long,Date,ConfirmedCases,Fatalities
0,1,Afghanistan,33.0,65.0,20200122,0.0,0.0
1,2,Afghanistan,33.0,65.0,20200123,0.0,0.0
2,3,Afghanistan,33.0,65.0,20200124,0.0,0.0
3,4,Afghanistan,33.0,65.0,20200125,0.0,0.0
4,5,Afghanistan,33.0,65.0,20200126,0.0,0.0


In [11]:
test = pd.read_csv("test.csv")   
test["Date"] = test["Date"].apply(lambda x: x.replace("-",""))
test["Date"]  = test["Date"].astype(int)

test["Lat"]  = test["Lat"].fillna(12.5211)
test["Long"]  = test["Long"].fillna(69.9683)
test.isnull().sum()

ForecastId           0
Province/State    6622
Country/Region       0
Lat                  0
Long                 0
Date                 0
dtype: int64

In [13]:
test["Date"].min() ## 2020-03-12
#date will contain no spaces

20200312

In [14]:
x = data[['Lat', 'Long', 'Date']]
y1 = data[['ConfirmedCases']]
y2 = data[['Fatalities']]
x_test = test[['Lat', 'Long', 'Date']]

In [15]:
from sklearn.tree import DecisionTreeClassifier
Tree_model = DecisionTreeClassifier(criterion='entropy')

In [16]:
##
Tree_model.fit(x,y1)
pred1 = Tree_model.predict(x_test)
pred1 = pd.DataFrame(pred1)
pred1.columns = ["ConfirmedCases_prediction"]

In [17]:
pred1.head()

,ConfirmedCases_prediction
0,7.0
1,7.0
2,7.0
3,7.0
4,7.0


In [18]:
##
Tree_model.fit(x,y2)
pred2 = Tree_model.predict(x_test)
pred2 = pd.DataFrame(pred2)
pred2.columns = ["Death_prediction"]

In [19]:
pred2.head()

,Death_prediction
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [21]:
Sub = pd.read_csv("submission.csv")
Sub.columns
sub_new = Sub[["ForecastId"]]

In [22]:
OP = pd.concat([pred1,pred2,sub_new],axis=1)
OP.head()
OP.columns = ['ConfirmedCases', 'Fatalities', 'ForecastId']
OP = OP[['ForecastId','ConfirmedCases', 'Fatalities']]

In [23]:
OP["ConfirmedCases"] = OP["ConfirmedCases"].astype(int)
OP["Fatalities"] = OP["Fatalities"].astype(int)

In [24]:
OP.head()

,ForecastId,ConfirmedCases,Fatalities
0,1,7,0
1,2,7,0
2,3,7,0
3,4,7,0
4,5,7,0


In [26]:
from sklearn.tree import DecisionTreeRegressor  
regressor = DecisionTreeRegressor(random_state = 0) 

In [27]:
regressor.fit(x,y1)
pred_r1 = regressor.predict(x_test)
pred_r1 = pd.DataFrame(pred_r1)
pred_r1.columns = ["ConfirmedCases_prediction"]

In [28]:
regressor.fit(x,y2)
pred_r2 = regressor.predict(x_test)
pred_r2 = pd.DataFrame(pred_r2)
pred_r2.columns = ["Death_prediction"]

In [29]:
OP_dr = pd.concat([sub_new,pred_r1,pred_r2],axis=1)
OP_dr.head()
OP_dr.columns = [ 'ForecastId','ConfirmedCases', 'Fatalities']

In [30]:
OP_dr["ConfirmedCases"] = OP_dr["ConfirmedCases"].astype(int)
OP_dr["Fatalities"] = OP_dr["Fatalities"].astype(int)
OP_dr.head()

,ForecastId,ConfirmedCases,Fatalities
0,1,7,0
1,2,7,0
2,3,7,0
3,4,7,0
4,5,7,0


In [31]:
OP_dr

,ForecastId,ConfirmedCases,Fatalities
0,1,7,0
1,2,7,0
2,3,7,0
3,4,7,0
4,5,7,0
...,...,...,...
12207,12208,0,0
12208,12209,0,0
12209,12210,0,0
12210,12211,0,0
